# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
## Lab 1.2 Object detection 데이터셋 준비하기 - RecordIO 파일 만들기

노트북을 처음 로딩할 때, Kernel로 **conda_mxnet_p36** 을 선택합니다.

### 랩 순서

1. GluonCV 설치하기
2. Ground Truth의 결과 manifest 파일로 부터 lst 파일 생성하기
3. im2rec 툴을 이용해서 RecordIO 파일 생성하기

## Task 1 - GluonCV 설치하기

In [ ]:
# 최초 실행시 GPU(p2/p3) instance에서는 아래 코드로 gluoncv 패키지를 설치하세요.
!pip install --upgrade mxnet-cu100mkl gluoncv
# CPU(c4/c5/m4/m5/t2/t3) instance에서는 아래 코드로 gluoncv 패키지를 설치하세요.
#!pip install --upgrade mxnet-mkl gluoncv

## Task 2 - Ground Truth의 결과 manifest 파일로 부터 lst 파일 생성하기

In [ ]:
import os
import subprocess

from gluoncv import utils
from gluoncv.data import LstDetection

from matplotlib import pyplot as plt

from utils import create_lst, write_lst_to_file

# <font color='red'>중요!!</font> 

## 아래 `gt_job_name` 변수의 값을 반드시 이전 단계에서 정의한 Ground Truth job의 이름으로 변경해주세요!!

In [ ]:
base_dir = '.'
im_dir = '{}/lab-images'.format(base_dir)
gt_manifest = '{}/output.manifest'.format(base_dir)

gt_job_name = 'devday-gluoncv-lab'

testset_ratio = 0.3

In [ ]:
lst_list, number_label, number_no_label = create_lst(gt_manifest, gt_job_name)

print('number of frames with label    : {}'.format(number_label))
print('number of frames with no label : {}'.format(number_no_label))

In [ ]:
testset_first_idx = int(len(lst_list) * (1-testset_ratio))

train_lst = lst_list[:testset_first_idx]
test_lst = lst_list[testset_first_idx:]

print('number of train dataset : {}'.format(len(train_lst)))
print('number of test dataset : {}'.format(len(test_lst)))

In [ ]:
train_lst_fname = '{}/train.lst'.format(base_dir)
test_lst_fname = '{}/test.lst'.format(base_dir)

write_lst_to_file(train_lst, train_lst_fname)
write_lst_to_file(test_lst, test_lst_fname)

In [ ]:
lst_dataset = LstDetection(test_lst_fname, root=im_dir)

img_idx = 1

img = lst_dataset[img_idx][0]

bbox=lst_dataset[img_idx][1][0][0:4].astype(int)
bbox = bbox.reshape(1,-1)

ids = lst_dataset[img_idx][1][0][4].astype(int)
ids = ids.reshape(1,-1)

In [ ]:
ax = utils.viz.plot_image(img)
plt.show()

In [ ]:
ax = utils.viz.plot_bbox(img, bbox, labels=ids) 

## Task 3 - im2rec 툴을 이용해서 RecordIO 파일 생성하기

In [ ]:
datasets = ['train', 'test']

for dataset in datasets:
    cmd = 'python im2rec.py {}.lst {} --no-shuffle --pass-through --pack-label'.format(dataset, im_dir)
    print(cmd)
    os.system(cmd)

In [ ]:
!ls -al *.rec

----
레이블링한 데이터로 RecordIO 파일이 만들어진 것을 확인할 수 있습니다. 뒤의 실습에서 이 형식의 파일이 직접 사용되는 것을 보시게 됩니다.

지금까지 Lab 1을 진행하시느라 수고하셨습니다.